In [1]:
import numpy as np
import pandas as pd
import subprocess

In [2]:
from IPython.display import clear_output

In [3]:
prqt_loc = '../../../../datasets/ess/parquet/'

In [4]:
p = subprocess.Popen(f'ls {prqt_loc}', shell=True, stdout=subprocess.PIPE)
ret = p.communicate()
print('Exit code: {}\nOutput:\n\n{}'.format(p.returncode, ret[0].decode('utf-8')))

Exit code: 0
Output:

ESS1_180115.parquet
ESS2_180115.parquet
ESS3_180115.parquet
ESS4_190213.parquet
ESS5_180115.parquet
ESS6_180115.parquet
ESS7_180115.parquet
ESS8_190213.parquet



In [5]:
parquets = [ pqt for pqt in ret[0].decode('utf-8').split('\n') if pqt.find('.parquet') > 0 ]

In [6]:
df_dtypes = pd.DataFrame()
for parquet in parquets:
    df = pd.read_parquet(f"{prqt_loc}{parquet}")
    dtype_dict = df.dtypes.to_dict()
    for key in dtype_dict:
        df_dtypes.loc[parquet,key] = dtype_dict[key]

In [7]:
vtype_order = {np.dtype('O'):4,np.dtype('float64'):3,np.dtype('int64'):1, np.nan: 2}

In [8]:
ess_schema_new = {}
for col in df_dtypes.columns:
    ess_schema_new[col] = sorted( list( set( df_dtypes[col].unique() ) ), key= lambda x: -vtype_order.get(x)  )[0]

In [9]:
def is_nan(x):
    return (x is np.nan or x != x)

In [10]:
for key in ess_schema_new.keys():
    if is_nan(ess_schema_new[key]):
        ess_schema_new[key] = np.dtype('float64')

In [11]:
for parquet in parquets:
    print(parquet)
    df = pd.read_parquet(f"{prqt_loc}{parquet}")
    diff_col = list( set(ess_schema_new.keys()) - set(df.columns) )
    df2 = pd.DataFrame()
    df2 = pd.DataFrame(columns = [col for col in diff_col])
    df2 = pd.concat([df,df2],sort=False)
    for col in list(ess_schema_new.keys()):
        df2[col] = df2[col].astype( ess_schema_new[col] , copy=False)
    df2.to_parquet(f"{prqt_loc}ns_{parquet}")

ESS1_180115.parquet
ESS2_180115.parquet
ESS3_180115.parquet
ESS4_190213.parquet
ESS5_180115.parquet
ESS6_180115.parquet
ESS7_180115.parquet
ESS8_190213.parquet
